<a href="https://colab.research.google.com/github/chasubeen/TalesRunnner/blob/doeun/koT5_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 저장 경로 설정
data_path = "/content/drive/MyDrive/7th-project/data"
model_path = "/content/drive/MyDrive/7th-project/model/koT5"

# **1. 데이터 준비**


## **1-1. 데이터 불러오기**
- 전처리 된 csv 파일 불러오기
- 해당 csv 파일에는 다음과 같은 정보들을 포함하고 있음

| Field | Description |
| --- | --- |
| id |* 이미지 → 캡션 정보와 annotation 정보를 결합 |
| imgpath | * `동화 삽화 생성 데이터 > 라벨링 데이터` 내의 jpg 파일 |
| caption | * BLIP을 통과시켜 얻어진 캡션|
| srcText | * LM의 output text|
| name | * 객체의 일반명사 (필수)|
| i_action | * 객체의 행동, 상태 (필수) |
| classification | * 의사소통/자연탐구/사회관계/예술경험/신체운동_건강 (필수) |
| character | * 캐릭터 (선택) |
| setting | * 셋팅 (선택) |
| action | * 행동 (선택) |
| feeling | * 감정 (선택) |
| causalRelationship | * 인과 (선택) |
| outcomeResolution | * 결과 (선택) |
| prediction | * 예측 (선택) |

In [ ]:
import pandas as pd

In [ ]:
## 전처리된 데이터(csv 파일) 불러오기
train_data = pd.read_csv(f"{data_path}/dataset_train.csv") # 학습
val_data = pd.read_csv(f"{data_path}/dataset_val.csv") # 검증

print(f"Train Data: {len(train_data)} samples")
print(f"Validation Data: {len(val_data)} samples")

Train Data: 40001 samples
Validation Data: 5000 samples


In [ ]:
train_data.head()

,id,img_path,caption,srcText,name,i_action,classification,character,setting,action,feeling,causalRelationship,outcomeResolution,prediction
0,03_05T_03S_9791157023899_42712,/content/data/TL_05T_신체운동_건강_03S_초...,"드레스와 모자를 쓴 세 명의 여성, 한 명은 모자를 쓰고 있습니다.","“플로렌스, 왜 리처드와 결혼을 하지 않겠다는 거니?” 어머니가 물었어요. 언니 파...",성인 여자,입에 한 손을 대고 놀란다,신체운동_건강,"플로렌스,어머니,파세노프,간호사",NaN,"물었어요,돌보며,되겠어요",NaN,NaN,NaN,NaN
1,03_05T_03S_9791157023998_39246,/content/data/TL_05T_신체운동_건강_03S_초...,책상에 앉아 있는 가족의 만화,"그래, 일본으로 가자. 호랑이 굴로 들어가 보는 거야.’ 일본에서 벗어나려면 일본을...",남성,펜을 잡고 있다.,신체운동_건강,"방정환,일본,어린이",동양 대학,"가자,들어가 보는,알아야,결심했어요,건너간,들어가,공부했어요,누려야",NaN,NaN,NaN,NaN
2,03_05T_03S_9791157023813_60463,/content/data/TL_05T_신체운동_건강_03S_초...,갈색 옷과 모자를 쓴 중국인,백 번을 죽고 또 죽어도 고려 임금을 향한 신의를 저버리지 않겠다는 뜻이었어요. 정...,성인 남자,모자를 썼다,신체운동_건강,"고려 임금,정몽주,이방원,자객",선죽교,보낸,NaN,NaN,비록 아쉽게 생을 마쳤지만 정몽주는 지금까지 충절을 지킨 가장 위대한 위인으로 기억...,NaN
3,03_05T_03S_9791157023981_34117,/content/data/TL_05T_신체운동_건강_03S_초...,한 여성이 남자와 개와 함께 텐트 앞에 앉아 있다,밀림을 떠나 케임브리지 대학에서 공부하고 왔는데도\n침팬지들은 제인을 잊지 않았던 ...,성인 여성,바나나를 들고 있다.,신체운동_건강,"침팬지들,제인,바나나,휴고","케임브리지 대학,밀림","떠나,공부하고,왔는데도,들어와,먹으며,놀려 댔어요,장난을 걸어요",NaN,NaN,NaN,NaN
4,03_05T_03S_9791157024247_33953,/content/data/TL_05T_신체운동_건강_03S_초...,여자가 컴퓨터가 있는 책상에 앉아 있다,"그런데 그날 저녁, 엄마가 뜻밖의 이야기를 하셨어요. ""승찬아, 이번 주 일요일은 ...",남자 아이,앉아 있다,신체운동_건강,"승찬이,엄마,할아버지,사촌 형,누나들","그날 저녁,일요일,토요일,하룻밤,저녁","하셨어요,하룻밤 자고,돌아올 거야,올 거래",NaN,NaN,NaN,NaN


In [ ]:
val_data.head()

,id,img_path,caption,srcText,name,i_action,classification,character,setting,action,feeling,causalRelationship,outcomeResolution,prediction
0,03_02T_03S_9788998212643_54879,/content/data/VL_02T_자연탐구_03S_초등_고하...,횡단보도로 길을 건너는 사람들의 집단,“출발합니다! 손잡이 꼭 잡으세요.” 삐이익. 다음 정류장에 내릴 손님이 벨을 눌렀...,신호등,초록색이다,자연탐구,"손잡이,정류장,손님,벨,차돌,신호등",NaN,"출발합니다,잡으세요,내릴,눌렀어요,멈췄다 가자,지키는,지켜야 해요",NaN,NaN,NaN,NaN
1,03_02T_03S_9788967760144_35692,/content/data/VL_02T_자연탐구_03S_초등_고하...,보트 위에 있는 남자와 여자의 그림,두 번째로 간 곳은 바다였다. “이번 모험은 고기잡이배에서 생활하는 것이다.” 마음...,고기잡이배,바다에 떠 있다,자연탐구,"모험,나는,그물,물고기,파도","바다,고기잡이배,하루 종일","출발했다,생활하는,간,태운,먹지 못하고,끌어 올려야",NaN,NaN,NaN,멀고 먼 바다로 출발했다.
2,03_02T_03S_9788998212636_5162,/content/data/VL_02T_자연탐구_03S_초등_고하...,들판에 있는 동물 무리의 그림,제주도에 여행 온 친구들은 힝힝 씨가 들려주는 제주도 이야기를 아주 좋아해요. “제...,해녀,바다에 있다,자연탐구,"친구,힝힝 씨,해녀,섬",제주도,들려주는,좋아해요,NaN,NaN,NaN
3,03_02T_03S_9791128211331_32163,/content/data/VL_02T_자연탐구_03S_초등_고하...,침대에서 자는 아이의 그림,쿠르쿠르는 에코스 덕분에 아주 오랜만에 단잠을 잘 수 있었어. 하지만 단 하루뿐이었...,난쟁이,침대에 누워있다,자연탐구,"쿠르쿠르,에코스",다음 날,"잘 수 있었어,들리기 시작한 거야",끔찍한,NaN,NaN,NaN
4,03_02T_03S_9791128211195_59242,/content/data/VL_02T_자연탐구_03S_초등_고하...,만화 캐릭터가 있는 빨간색과 주황색 배경,“이 원반은 사람의 핏속에 있는 적혈구랍니다. 적혈구는 사람의 몸속 구석구석으로 산...,적혈구,붉은색 원반 모양을 하고 흘러가고 있어요.,자연탐구,"원반,사람,적혈구,산소",핏속,"나르지요,여행할 거예요",NaN,NaN,NaN,NaN


## **1-2. text 전처리**





### **input text**
- special token을 활용하여 각 정보를 구분
  - 필수 필드
    - 결측치 확인(결측치 허용 x)
  - 선택 필드
    - 다중 아이템의 경우 (,)로 구분 후 각 항목에 토큰 부여
    - 결측치가 있는 경우 `<empty>` 토큰 부여
- caption과 보조적 정보를 포함하여 구성
  - 행별 고유 시드 설정: 각 행의 id(또는 index)를 seed로 설정   
  → 동일한 데이터를 사용하면 언제나 같은 결과가 나오도록 보장
  - 랜덤 순서 보장: 각 행마다 token의 순서를 랜덤하게 설정  
  → 모델이 token 간 순서나 위치에 지나치게 의존하지 않고, 각 요소의 의미와 역할을 학습하도록 유도

In [ ]:
import random
import hashlib

In [ ]:
### Input Text 생성 함수

def generate_input_text(row):
    ## 고유 seed 설정(해시 기반)
    seed_value = int(hashlib.md5(str(row["id"]).encode()).hexdigest(), 16) % (10 ** 8)
    random.seed(seed_value)


    ## 필수 필드
    # 존재성 보장(assertion)
    required_fields = ["caption", "name", "i_action", "classification"]
    for field in required_fields:
        assert pd.notna(row[field]) and row[field].strip(), f"Error: '{field}' 필드는 비워둘 수 없습니다."

    required_fields = [
        f"<caption> {row['caption'].strip()}",
        f"<name> {row['name'].strip()}",
        f"<i_action> {row['i_action'].strip()}",
        f"<classification> {row['classification'].strip()}"
    ]


    ## 선택 필드
    optional_fields = {
        "character": "<character>",
        "setting": "<setting>",
        "action": "<action>",
        "feeling": "<feeling>",
        "causalRelationship": "<causalRelationship>",
        "outcomeResolution": "<outcomeResolution>",
        "prediction": "<prediction>"
    }

    optional_tokens = []
    for field, token in optional_fields.items():
        if pd.notna(row[field]) and row[field].strip():
            # 여러 항목 처리(쉼표로 구분)
            items = [item.strip() for item in row[field].split(",")]
            optional_tokens.extend([f"{token} {item}" for item in items])
        else:
            optional_tokens.append(f"{token} <empty>")


    ## 토큰 순서 섞기
    all_tokens = required_fields + optional_tokens
    random.shuffle(all_tokens)


    return "generate: " + " ".join(all_tokens) # prefix 추가

In [ ]:
## Input Text 생성
train_data["input_text"] = train_data.apply(generate_input_text, axis = 1)
val_data["input_text"] = val_data.apply(generate_input_text, axis = 1)

### **output text**

In [ ]:
import re

In [ ]:
### Output Text 전처리 함수

def preprocess_output_text(story):
    if pd.isna(story):
        return "<empty>"

    story = re.sub(r'[^\w\s가-힣.,?!]', '', story) # 특수 문자 제거(문장 부호 유지)

    story = re.sub(r'\s+', ' ', story).strip() # 불필요한 공백 제거

    return story

In [ ]:
## Output Text 전처리

train_data["output_text"] = train_data["srcText"].apply(preprocess_output_text)
val_data["output_text"] = val_data["srcText"].apply(preprocess_output_text)

**데이터 확인**  

In [ ]:
train_data[["id", "input_text", "output_text"]].head()

,id,input_text,output_text
0,03_05T_03S_9791157023899_42712,generate: <setting> <empty> <prediction> <empt...,"플로렌스, 왜 리처드와 결혼을 하지 않겠다는 거니? 어머니가 물었어요. 언니 파세노..."
1,03_05T_03S_9791157023998_39246,generate: <action> 건너간 <name> 남성 <action> 누려야 ...,"그래, 일본으로 가자. 호랑이 굴로 들어가 보는 거야. 일본에서 벗어나려면 일본을 ..."
2,03_05T_03S_9791157023813_60463,generate: <outcomeResolution> 비록 아쉽게 생을 마쳤지만 정...,백 번을 죽고 또 죽어도 고려 임금을 향한 신의를 저버리지 않겠다는 뜻이었어요. 정...
3,03_05T_03S_9791157023981_34117,generate: <action> 장난을 걸어요 <character> 휴고 <cap...,밀림을 떠나 케임브리지 대학에서 공부하고 왔는데도 침팬지들은 제인을 잊지 않았던 거...
4,03_05T_03S_9791157024247_33953,generate: <character> 승찬이 <prediction> <empty>...,"그런데 그날 저녁, 엄마가 뜻밖의 이야기를 하셨어요. 승찬아, 이번 주 일요일은 할..."


In [ ]:
val_data[["id", "input_text", "output_text"]].head()

,id,input_text,output_text
0,03_02T_03S_9788998212643_54879,generate: <character> 손님 <action> 눌렀어요 <charac...,출발합니다! 손잡이 꼭 잡으세요. 삐이익. 다음 정류장에 내릴 손님이 벨을 눌렀어요...
1,03_02T_03S_9788967760144_35692,generate: <character> 나는 <caption> 보트 위에 있는 남자...,두 번째로 간 곳은 바다였다. 이번 모험은 고기잡이배에서 생활하는 것이다. 마음의 ...
2,03_02T_03S_9788998212636_5162,generate: <classification> 자연탐구 <character> 힝힝...,제주도에 여행 온 친구들은 힝힝 씨가 들려주는 제주도 이야기를 아주 좋아해요. 제주...
3,03_02T_03S_9791128211331_32163,generate: <i_action> 침대에 누워있다 <prediction> <em...,쿠르쿠르는 에코스 덕분에 아주 오랜만에 단잠을 잘 수 있었어. 하지만 단 하루뿐이었...
4,03_02T_03S_9791128211195_59242,generate: <causalRelationship> <empty> <outcom...,이 원반은 사람의 핏속에 있는 적혈구랍니다. 적혈구는 사람의 몸속 구석구석으로 산소...


# **2. Modeling**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json

## **2-1. model 설정**
- pre-trained `tokenizer`와 `model` 불러오기
- 한국어 말뭉치로 훈련된 koT5 활용
  - 현재는 **base** 활용
  - small version도 활용 가능

  ```python
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

  tokenizer = AutoTokenizer.from_pretrained("wisenut-nlp-team/KoT5-small")
  model = AutoModelForSeq2SeqLM.from_pretrained("wisenut-nlp-team/KoT5-small")
  ```

### **model file 구성**
1. **`config.json`**:
   - 모델 설정 파일
    - hidden size, layer 개수, attention heads 등
   - 모델 아키텍처와 관련된 정보를 포함

2. **`generation_config.json`**:
   - 텍스트 생성 시의 기본 설정값
    - `max_length`, `num_beams`, `repetition_penalty` 등

3. **`model.safetensors`**:
   - 모델의 가중치(weight)를 저장한 파일
    - `safetensors`는 더 안전하고 빠른 저장 형식을 사용

4. **`special_tokens_map.json`**:
   - 추가된 `special token`들의 정보를 명시적으로 저장
5. **`tokenizer_config.json`**:
   - `tokenizer`의 일반 설정
    - 사용된 vocabulary 크기, 특수 토큰 정보, tokenizer 클래스 이름 등을 저장
6. **`tokenizer.json`**:
   - `tokenizer`의 전체 단어 사전(vocabulary)과 매핑을 포함
   - 이 파일에 추가된 `special token`들이 반영되어, 새로운 토큰이 vocabulary에 추가된 것을 확인 가능

In [ ]:
## 모델 및 Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained("wisenut-nlp-team/KoT5-base", revision = "paraphrase")
model = AutoModelForSeq2SeqLM.from_pretrained("wisenut-nlp-team/KoT5-base", revision = "paraphrase")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

## **2-2. Tokenizer 설정**
- special token 추가 및 model, tokenizer 확장

In [ ]:
## 특수 token 정의
special_tokens = [
    "<caption>", "<name>", "<i_action>", "<classification>", "<character>",
    "<setting>", "<action>", "<feeling>", "<causalRelationship>",
    "<outcomeResolution>", "<prediction>", "<empty>"
]

In [ ]:
## 특수 token 확장
# 1. 기존 특수 토큰 가져오기
existing_special_tokens = tokenizer.special_tokens_map.get("additional_special_tokens", [])

# 2. 특수 토큰 병합 (중복 제거)
merged_special_tokens = list(set(existing_special_tokens + special_tokens))

# 3. 토크나이저에 특수 토큰 추가
tokenizer.add_special_tokens({'additional_special_tokens': merged_special_tokens})

# 4. 모델 Embedding 크기 재조정
model.resize_token_embeddings(len(tokenizer))

# 5. 모델과 토크나이저 저장
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

In [ ]:
print("Special tokens added:", tokenizer.special_tokens_map)
print("Vocabulary size:", len(tokenizer))

special_token_ids = tokenizer.convert_tokens_to_ids(special_tokens)
print("Special Token IDs:", sorted(special_token_ids))

Special tokens added: {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_2>', '<extra_id_64>', '<extra_id_8>', '<extra_id_15>', '<character>', '<extra_id_7>', '<extra_id_67>', '<extra_id_38>', '<extra_id_30>', '<extra_id_69>', '<extra_id_14>', '<prediction>', '<extra_id_16>', '<extra_id_35>', '<extra_id_40>', '<extra_id_82>', '<extra_id_59>', '<extra_id_26>', '<extra_id_79>', '<extra_id_0>', '<extra_id_36>', '<extra_id_41>', '<extra_id_9>', '<extra_id_72>', '<extra_id_80>', '<extra_id_57>', '<extra_id_74>', '<setting>', '<extra_id_93>', '<extra_id_23>', '<extra_id_96>', '<action>', '<extra_id_4>', '<extra_id_11>', '<extra_id_77>', '<extra_id_87>', '<extra_id_43>', '<extra_id_88>', '<extra_id_60>', '<extra_id_99>', '<extra_id_71>', '<feeling>', '<outcomeResolution>', '<extra_id_12>', '<extra_id_37>', '<extra_id_62>', '<causalRelationship>', '<extra_id_18>', '<extra_id_63>', '<extra_id_32>', '<name>', '<extra_id_53>', '<extra_id_68>

## **2-3. Dataset 및 DataLoader 구성**
- model을 학습시키기 위한 데이터셋 구축 및 토큰화

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorForSeq2Seq

In [ ]:
### masking 함수 정의

def special_token_masking(input_ids, attention_mask, special_token_ids):
    mask_indices = (input_ids.unsqueeze(-1) == torch.tensor(special_token_ids)).any(dim=-1)
    attention_mask[mask_indices] = 0

    return attention_mask

In [ ]:
### Dataset 클래스 정의

class StoryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, special_token_ids, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.special_token_ids = special_token_ids

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        input_encoding = self.tokenizer(
            row["input_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        target_encoding = self.tokenizer(
            row["output_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = input_encoding["input_ids"].squeeze(0)
        attention_mask = input_encoding["attention_mask"].squeeze(0)
         # Special Token 마스킹 적용
        attention_mask = special_token_masking(input_ids, attention_mask, self.special_token_ids)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": target_encoding["input_ids"].squeeze(0),
        }

In [ ]:
## Dataset 생성

train_dataset = StoryDataset(train_data, tokenizer, special_token_ids)
val_dataset = StoryDataset(val_data, tokenizer, special_token_ids)

In [ ]:
## DataLoader 생성

# Dynamic padding 적용
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

train_dataloader = DataLoader(train_dataset, batch_size = 8, shuffle = True, collate_fn = data_collator)
val_dataloader = DataLoader(val_dataset, batch_size = 8, collate_fn = data_collator)

## **2-4. 샘플 확인**
- special token에 대한 처리가 제대로 진행되고 있는지 확인

In [ ]:
# Tokenizer가 특수 토큰을 정확히 인식하는지 확인
for token in special_tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"Token: {token}, Token ID: {token_id}, Decoded: {tokenizer.decode([token_id])}")

Token: <caption>, Token ID: 32108, Decoded: <caption>
Token: <name>, Token ID: 32107, Decoded: <name>
Token: <i_action>, Token ID: 32109, Decoded: <i_action>
Token: <classification>, Token ID: 32110, Decoded: <classification>
Token: <character>, Token ID: 32100, Decoded: <character>
Token: <setting>, Token ID: 32102, Decoded: <setting>
Token: <action>, Token ID: 32103, Decoded: <action>
Token: <feeling>, Token ID: 32104, Decoded: <feeling>
Token: <causalRelationship>, Token ID: 32106, Decoded: <causalRelationship>
Token: <outcomeResolution>, Token ID: 32105, Decoded: <outcomeResolution>
Token: <prediction>, Token ID: 32101, Decoded: <prediction>
Token: <empty>, Token ID: 32111, Decoded: <empty>


In [ ]:
## Tokenization 상세 확인

# 샘플 배치 확인
sample_batch = next(iter(train_dataloader))
input_ids = sample_batch['input_ids']
attention_mask = sample_batch['attention_mask']

for i in range(4):  # 배치 내 sample 확인
    text = sample_batch['input_ids'][i]
    tokens = tokenizer.convert_ids_to_tokens(text)
    mask = sample_batch['attention_mask'][i].tolist()

    print(f"Decoded Text {i+1}: {tokenizer.decode(text, skip_special_tokens=False)}")
    print(f"Tokens: {tokens}")
    print(f"Attention Mask: {mask}")
    print("=" * 50)

Decoded Text 1: generate: <caption> 자동차와 남자와 여자 사이에 대화하는 가족의 만화 <setting> 제주도 <setting> 사이언스 빌리지 <character> 아빠 <action> 함께 갔어요. <classification> 자연탐구 <i_action> 시끌벅적하다 <character> 유비 <character> 황박사님 <action> 힐끔힐끔 쳐다보고 있었어요 <character> 호텔 <prediction> <empty> <character> 진달래 이모 <causalRelationship> <empty> <character> 마리 <outcomeResolution> <empty> <name> 마을 사람들 <character> 마을 <character> 집 <feeling> 좋아한다는 <character> 엄마 <character> 할머니 <action> 여행을 떠났어요</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Tokens: ['▁g', 'en', 'er', 'ate', ':', '▁', '<caption>', '▁자동차', '와', '▁남자와', '▁여자', '▁사이에', '▁대화', '하는', '▁가족의', '▁만화', '▁', '<setting>', '▁제주도', '▁', '<setting>', '▁사이', '언스', '▁빌', '리지', '▁', '<character>', '▁아빠', '▁', '<action>', '▁함께', '▁갔', '어요', '.', '▁', '<classification>', '▁자연', '탐', '구', '▁', '<i_action>', '▁시', '끌', '벅', '적', '하다', '▁', '<character>', '▁유', '비', '▁', '<character>', '▁황', '박사', '님', '▁', '<action>', '▁힐', '끔', '힐', '끔', '▁쳐다

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [ ]:
labels_batch = sample_batch['labels'][0]

decoded_batch = tokenizer.decode(
    labels_batch,
    skip_special_tokens=True
)

print("✅ [배치 디코딩 결과]:")
print(decoded_batch)

✅ [배치 디코딩 결과]:
사이언스 빌리지가 만들어진 지 100년이 되어 여러 가지 축제 행사 때문에 일주일 동안 온 마을이 시끌벅적했어요. 너희들 할머니랑 집 잘 보고 있어라! 유비와 마리, 그리고 투덜이네 엄마 아빠는 모두 제주도로 여행을 떠났어요. 제주도의 한 호텔에서 사이언스 빌리지의 미래를 위해 회의도 하고, 회의가 끝난 후에는 관광도 한다고 했어요. 이번 여행에는 황박사님과 진달래 이모도 함께 갔어요. 여행을 떠날 때 보니 황박사님이 자꾸 진달래 이모를 힐끔힐끔 쳐다보고 있었어요. 황박사님이 진달래 이모를 좋아한다는 소문이 사실인가 봐요.


In [ ]:
## 특수 토큰 전후 확인

for i in range(4):
    tokens = tokenizer.convert_ids_to_tokens(input_ids[i])
    attention = attention_mask[i].tolist()

    for idx, (token, mask) in enumerate(zip(tokens, attention)):
        if token in special_tokens or token == "<empty>":
            print(f"Token: {token}, Mask: {mask}, Prev: {attention[idx-1] if idx > 0 else 'N/A'}, Next: {attention[idx+1] if idx < len(tokens)-1 else 'N/A'}")
    print("="*50)

Token: <caption>, Mask: 0, Prev: 1, Next: 1
Token: <setting>, Mask: 0, Prev: 1, Next: 1
Token: <setting>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <action>, Mask: 0, Prev: 1, Next: 1
Token: <classification>, Mask: 0, Prev: 1, Next: 1
Token: <i_action>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <action>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <prediction>, Mask: 0, Prev: 1, Next: 1
Token: <empty>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <causalRelationship>, Mask: 0, Prev: 1, Next: 1
Token: <empty>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <outcomeResolution>, Mask: 0, Prev: 1, Next: 1
Token: <empty>, Mask: 0, Prev: 1, Next: 1
Token: <name>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Nex

# **3. Training**

## 3-1. 라이브러리 설치 및 import

In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
import os
import gc
from transformers import get_scheduler
from tqdm.auto import tqdm
import torch
from torch.optim import AdamW
import evaluate

In [ ]:
## 장치 설정 및 모델 확인

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

## 3-2. 하이퍼파라미터 튜닝

- Bayesian Optimization 을 사용하여 최적의 하이퍼파라미터 조합을 찾기
- `learning_rate`, `warmup_steps`, `patience`, `epochs`를 탐색 범위로 설정

In [ ]:
## 하이퍼파라미터 튜닝

import torch
import gc
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from transformers import AutoModel, AutoTokenizer

# GPU 설정 (CUDA 사용 가능 여부 확인)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 샘플 데이터셋 사용
train_data_sample = train_data.sample(n=2000, random_state=42)  # 원본 데이터에서 2000개 샘플링
val_data_sample = val_data.sample(n=500, random_state=42)  # 검증 데이터는 500개 샘플링

# 데이터셋 및 데이터로더 생성
train_dataset = StoryDataset(train_data_sample, tokenizer, special_token_ids)
val_dataset = StoryDataset(val_data_sample, tokenizer, special_token_ids)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(val_dataset, batch_size=8, collate_fn=data_collator)


search_space = [
    Real(1e-5, 5e-4, name="learning_rate"),  # 학습률 (10^-5 ~ 5x10^-4)
    Integer(1, 5, name="warmup_steps"),  # 워밍업 스텝 (1~5 비율 적용)
    Integer(2, 5, name="patience"),  # Early Stopping Patience (2~5)
    Integer(5, 20, name="epochs") # Epoch 범위(5~20)
]


# 평가 함수 정의 (Validation Loss 기준 최적화)
@use_named_args(search_space)
def objective(learning_rate, warmup_steps, patience, epochs):
    print(f"\n=== Evaluating: lr={learning_rate}, warmup_steps={warmup_steps}, patience={patience}, epochs={epochs} ===")

    # 저장된 토크나이저와 모델을 로드
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    # 스케줄러 설정
    num_training_steps = epochs * len(train_dataloader)
    scheduler = get_scheduler(
        "linear", optimizer=optimizer, num_warmup_steps=int(warmup_steps * num_training_steps / 10),
        num_training_steps=num_training_steps
    )

    best_val_loss = float("inf")
    early_stop_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_dataloader)

        # Validation Loss 계산
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.item()

        avg_val_loss = val_loss / len(val_dataloader)
        print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Validation Loss={avg_val_loss:.4f}")

        # Early Stopping 적용
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}. Best val_loss={best_val_loss:.4f}")
                break

    # GPU 캐시 정리
    gc.collect()
    torch.cuda.empty_cache()

    return best_val_loss  # 최소화할 값 반환

# Bayesian Optimization 실행 (10번 탐색)
best_result = gp_minimize(objective, search_space, n_calls=10)
print("\n=== 최적 하이퍼파라미터 조합 ===")
print(f"Learning Rate: {best_result.x[0]:.6f}")
print(f"Warmup Steps Ratio: {best_result.x[1]}")
print(f"Early Stopping Patience: {best_result.x[2]}")
print(f"Optimal Epochs: {best_result.x[3]}")

```
Using device: cuda

=== Evaluating: lr=0.0001063930053704756, warmup_steps=3, patience=4, epochs=13 ===
Epoch 1: Train Loss=5.4076, Validation Loss=1.6179
Epoch 2: Train Loss=1.6246, Validation Loss=1.4301
Epoch 3: Train Loss=1.4648, Validation Loss=1.3553
Epoch 4: Train Loss=1.3750, Validation Loss=1.3202
Epoch 5: Train Loss=1.2969, Validation Loss=1.2921
Epoch 6: Train Loss=1.2306, Validation Loss=1.2790
Epoch 7: Train Loss=1.1775, Validation Loss=1.2764
Epoch 8: Train Loss=1.1289, Validation Loss=1.2755
Epoch 9: Train Loss=1.0922, Validation Loss=1.2772
Epoch 10: Train Loss=1.0583, Validation Loss=1.2798
Epoch 11: Train Loss=1.0310, Validation Loss=1.2872
Epoch 12: Train Loss=1.0140, Validation Loss=1.2877
⏹Early stopping at epoch 12. Best val_loss=1.2755

=== Evaluating: lr=0.00042089666560782914, warmup_steps=4, patience=3, epochs=10 ===
Epoch 1: Train Loss=3.6228, Validation Loss=1.4366
Epoch 2: Train Loss=1.4497, Validation Loss=1.3254
Epoch 3: Train Loss=1.3094, Validation Loss=1.2948
Epoch 4: Train Loss=1.1958, Validation Loss=1.2747
Epoch 5: Train Loss=1.0688, Validation Loss=1.2831
Epoch 6: Train Loss=0.9509, Validation Loss=1.3124
Epoch 7: Train Loss=0.8400, Validation Loss=1.3529
⏹Early stopping at epoch 7. Best val_loss=1.2747

=== Evaluating: lr=0.00027278045852754463, warmup_steps=4, patience=4, epochs=19 ===
Epoch 1: Train Loss=4.8405, Validation Loss=1.5699
Epoch 2: Train Loss=1.5810, Validation Loss=1.4010
Epoch 3: Train Loss=1.4254, Validation Loss=1.3368
Epoch 4: Train Loss=1.3326, Validation Loss=1.3014
Epoch 5: Train Loss=1.2530, Validation Loss=1.2803
Epoch 6: Train Loss=1.1672, Validation Loss=1.2718
Epoch 7: Train Loss=1.0825, Validation Loss=1.2787
Epoch 8: Train Loss=1.0100, Validation Loss=1.3004
Epoch 9: Train Loss=0.9188, Validation Loss=1.3255
Epoch 10: Train Loss=0.8407, Validation Loss=1.3480
⏹Early stopping at epoch 10. Best val_loss=1.2718

=== Evaluating: lr=0.0004688449695617386, warmup_steps=2, patience=4, epochs=7 ===
Epoch 1: Train Loss=2.8023, Validation Loss=1.3503
Epoch 2: Train Loss=1.3431, Validation Loss=1.2895
Epoch 3: Train Loss=1.1681, Validation Loss=1.2688
Epoch 4: Train Loss=1.0286, Validation Loss=1.2848
Epoch 5: Train Loss=0.9175, Validation Loss=1.3120
Epoch 6: Train Loss=0.8361, Validation Loss=1.3392
Epoch 7: Train Loss=0.7735, Validation Loss=1.3676
⏹Early stopping at epoch 7. Best val_loss=1.2688

=== Evaluating: lr=0.00043747491827152596, warmup_steps=3, patience=5, epochs=20 ===
Epoch 1: Train Loss=4.0318, Validation Loss=1.4729
Epoch 2: Train Loss=1.4858, Validation Loss=1.3504
Epoch 3: Train Loss=1.3524, Validation Loss=1.2928
Epoch 4: Train Loss=1.2369, Validation Loss=1.2779
Epoch 5: Train Loss=1.1359, Validation Loss=1.2804
Epoch 6: Train Loss=1.0380, Validation Loss=1.2957
Epoch 7: Train Loss=0.9376, Validation Loss=1.3304
Epoch 8: Train Loss=0.8177, Validation Loss=1.3558
Epoch 9: Train Loss=0.7165, Validation Loss=1.4206
⏹Early stopping at epoch 9. Best val_loss=1.2779

=== Evaluating: lr=0.0002791161202725361, warmup_steps=2, patience=5, epochs=8 ===
Epoch 1: Train Loss=3.1949, Validation Loss=1.3963
Epoch 2: Train Loss=1.3961, Validation Loss=1.2978
Epoch 3: Train Loss=1.2451, Validation Loss=1.2676
Epoch 4: Train Loss=1.1342, Validation Loss=1.2679
Epoch 5: Train Loss=1.0377, Validation Loss=1.2763
Epoch 6: Train Loss=0.9659, Validation Loss=1.2912
Epoch 7: Train Loss=0.9178, Validation Loss=1.3035
Epoch 8: Train Loss=0.8775, Validation Loss=1.3151
⏹Early stopping at epoch 8. Best val_loss=1.2676

=== Evaluating: lr=0.00040631420119938534, warmup_steps=2, patience=2, epochs=9 ===
Epoch 1: Train Loss=3.0638, Validation Loss=1.3839
Epoch 2: Train Loss=1.3830, Validation Loss=1.2982
Epoch 3: Train Loss=1.2217, Validation Loss=1.2705
Epoch 4: Train Loss=1.0829, Validation Loss=1.2808
Epoch 5: Train Loss=0.9718, Validation Loss=1.3024
⏹Early stopping at epoch 5. Best val_loss=1.2705

=== Evaluating: lr=0.00030264247693827673, warmup_steps=2, patience=3, epochs=9 ===
Epoch 1: Train Loss=3.2385, Validation Loss=1.4113
Epoch 2: Train Loss=1.4167, Validation Loss=1.3153
Epoch 3: Train Loss=1.2678, Validation Loss=1.2838
Epoch 4: Train Loss=1.1447, Validation Loss=1.2775
Epoch 5: Train Loss=1.0483, Validation Loss=1.2853
Epoch 6: Train Loss=0.9685, Validation Loss=1.3079
Epoch 7: Train Loss=0.9006, Validation Loss=1.3258
⏹Early stopping at epoch 7. Best val_loss=1.2775

=== Evaluating: lr=0.00022543788568672894, warmup_steps=4, patience=3, epochs=5 ===
Epoch 1: Train Loss=3.5282, Validation Loss=1.4271
Epoch 2: Train Loss=1.4408, Validation Loss=1.3238
Epoch 3: Train Loss=1.2953, Validation Loss=1.2787
Epoch 4: Train Loss=1.1902, Validation Loss=1.2719
Epoch 5: Train Loss=1.1229, Validation Loss=1.2761

=== Evaluating: lr=0.0001654940801821774, warmup_steps=2, patience=2, epochs=15 ===
Epoch 1: Train Loss=4.3706, Validation Loss=1.5088
Epoch 2: Train Loss=1.5290, Validation Loss=1.3759
Epoch 3: Train Loss=1.3906, Validation Loss=1.3195
Epoch 4: Train Loss=1.2871, Validation Loss=1.2858
Epoch 5: Train Loss=1.1982, Validation Loss=1.2805
Epoch 6: Train Loss=1.1256, Validation Loss=1.2765
Epoch 7: Train Loss=1.0609, Validation Loss=1.2815
Epoch 8: Train Loss=0.9996, Validation Loss=1.2943
⏹Early stopping at epoch 8. Best val_loss=1.2765

=== 최적 하이퍼파라미터 조합 ===
Learning Rate: 0.000279
Warmup Steps Ratio: 2
Early Stopping Patience: 5
Optimal Epochs: 8
```

## 3-3. 최적 하이퍼파라미터를 사용한 모델 학습 및 저장

- Bayesian Optimization 으로 찾은 최적 하이퍼파라미터를 사용하여 모델 학습
- 체크포인트 저장
- Early Stopping 적용
- 최종 모델 및 토크나이저 저장

In [ ]:
from transformers import AutoModel

In [ ]:
# 최적 하이퍼파라미터
learning_rate = 0.000279
warmup_steps_ratio = 2
patience = 5
epochs = 8

# 체크포인트 및 최종 모델 저장 경로
checkpoint_path = "/content/drive/MyDrive/7th-project/model/kot5_checkpoint_final.pt"
final_model_path = "/content/drive/MyDrive/7th-project/model/koT5/best_model2"

# 모델 및 옵티마이저 초기화
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 학습률 스케줄러 설정
num_training_steps = epochs * len(train_dataloader)
num_warmup_steps = int(warmup_steps_ratio * num_training_steps / 10)  # 전체 학습 스텝의 비율 적용
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Early Stopping 설정
best_val_loss = float("inf")
early_stop_counter = 0

start_epoch = 0

# 이전 체크포인트가 있다면 불러오기
if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint from {checkpoint_path} ...")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    # 모델의 임베딩 크기 조정 (토크나이저 크기와 맞추기)
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint["model_state_dict"], strict=False) # strict=False 옵션 추가
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    best_val_loss = checkpoint["best_val_loss"]
    early_stop_counter = checkpoint["early_stop_counter"]
    start_epoch = checkpoint["epoch"] + 1  # 다음 epoch부터 학습 시작
    print(f"Resuming training from epoch {start_epoch}")


model.to(device)

# 학습 시작
for epoch in range(start_epoch, epochs):
    model.train()
    train_loss = 0.0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        # Gradient Clipping 추가
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataloader)

    # 검증 과정
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Validation Loss={avg_val_loss:.4f}")

    # Early Stopping 적용
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        # 가장 낮은 validation loss 기준으로 모델 저장
        torch.save(model.state_dict(), "best_model.pt")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}. Best val_loss={best_val_loss:.4f}")
            break

    # 체크포인트 저장
    checkpoint = {
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "best_val_loss": best_val_loss,
        "early_stop_counter": early_stop_counter,
    }
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch+1}")

# 최종 모델 및 토크나이저 저장
print("Saving final model and tokenizer...")
tokenizer.save_pretrained(final_model_path)
model.save_pretrained(final_model_path)

# GPU 캐시 정리
gc.collect()
torch.cuda.empty_cache()

print("Training complete. Best Validation Loss:", best_val_loss)
print(f"Final model and tokenizer saved at {final_model_path}")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1: Train Loss=1.6362, Validation Loss=1.1139
Checkpoint saved at epoch 1
Epoch 2: Train Loss=1.1716, Validation Loss=1.0535
Checkpoint saved at epoch 2
Epoch 3: Train Loss=1.0504, Validation Loss=1.0049
Checkpoint saved at epoch 3
Epoch 4: Train Loss=0.9431, Validation Loss=0.9656
Checkpoint saved at epoch 4
Epoch 5: Train Loss=0.8553, Validation Loss=0.9488
Checkpoint saved at epoch 5
Epoch 6: Train Loss=0.7840, Validation Loss=0.9341
Checkpoint saved at epoch 6
Epoch 7: Train Loss=0.7264, Validation Loss=0.9297
Checkpoint saved at epoch 7
Epoch 8: Train Loss=0.6855, Validation Loss=0.9320
Checkpoint saved at epoch 8
Saving final model and tokenizer...
Training complete. Best Validation Loss: 0.9297051637649536
Final model and tokenizer saved at /content/drive/MyDrive/7th-project/model/koT5/best_model2
